<hr>

In [189]:
import os
os.getcwd()

'C:\\Users\\Pranav\\git\\evomorph\\sandbox'

In [190]:
df = pd.read_csv("2020-10-15-hexGRN_sweep/mean_wave_velocity.csv", index_col=0)
mdf = pd.read_csv("2020-10-15-hexGRN_sweep/metadata.csv", index_col=0)

In [197]:
df["Mean wavefront speed"].values.argmax()

1837

In [198]:
df.iloc[1837]

promoter stregth          3.000000
signaling threshold       0.010000
feedback strength         0.000000
% shared interface      100.000000
delay time                0.100000
Mean wavefront speed      8.729218
Name: 1837, dtype: float64